We assume that you have completed the first part of `LAB140 - Learn how to build a modern data stack in an hour ` - Data Integration, which involved creating your first lakehouse using Copy Data, as well as copying the data to the `nyc_taxi` table. 

[Sharing a link to the data used in the lab.](https://github.com/ekote/azure-architect#:~:text=part%2D00175%2Dtid%2D4753095944193949832%2Dfee7e113%2D666d%2D4114%2D9fcb%2Dbcd3046479f3%2D2745%2D1.c000.parquet)

# 1. Access the lakehouse

The foundation of Project Trident is a Lakehouse, which is built on top of the OneLake scalable storage layer and uses Apache Spark and SQL compute engines for big data processing. A Lakehouse is a unified platform that combines:
- The flexible and scalable storage of a data lake
- The ability to query and analyze data of a data warehouse


Imagine your company has been storing structured data from NYC Taxi's transactional system, such as trip history, passenger counts, and fare information in a data warehouse. However, you have also collected unstructured data from social media, website logs, and third-party sources related to NYC Taxi, which are difficult to manage and analyze using the existing data warehouse infrastructure.

Your company's new directive is to improve its decision-making capabilities by analyzing data in various formats across multiple sources. Therefore, the company decides to leverage Project Trident's capabilities to analyze and manage these diverse datasets more efficiently.

**Let's discover the lakehouse!**

<mark>Now, please follow along with the instructor.
</mark>


In [ ]:
df = spark.sql("SELECT * FROM lab140lakehouse.nyc_taxi LIMIT 1000")
display(df)

# 2. Lakehouse artifact overview

A Lakehouse presents as a database and is built on top of a data lake using Delta Lake files and tables. Lakehouses combine the SQL-based analytical capabilities of a relational data warehouse and the flexibility and scalability of a data lake. Lakehouses store all data formats and can be used with various analytics tools and programming languages. As cloud-based solutions, lakehouses can scale automatically and provide high availability and disaster recovery.

Some benefits of a lakehouse include:
- Lakehouses use Spark and SQL engines to process large-scale data and support machine learning or predictive modeling analytics.
- Lakehouse data is organized in a schema-on-read format, which means you define the schema as needed rather than having a predefined schema.
- Lakehouses support ACID (Atomicity, Consistency, Isolation, Durability) transactions through Delta Lake formatted tables for data consistency and integrity.
- Lakehouses are a single location for data engineers, data scientists, and data analysts to access and use data.

A Lakehouse is a great option if you want a scalable analytics solution that maintains data consistency. 



<mark>Now, please follow along with the instructor.
</mark>



# 3. Get data from the lakehouse

The most common way to work with data in delta tables in Spark is to use Spark SQL. You can embed SQL statements in other languages (such as PySpark or Scala) by using the spark.sql library.

<mark>Now, please follow along with the instructor.
</mark>

There are three ways to access Lakehouse data in your notebook:

* Using a relative path to the default Lakehouse, as shown in the code example.
* Using the full ABFS (Azure Blob File System) path.
* Using the syntax "/lakehousename/path", which is only valid when accessing a Lakehouse within the same workspace.




`df = spark.sql("SELECT * FROM lab140lakehouse.nyc_taxi LIMIT 1000")` - This line of code uses the `spark.sql()` function to run an SQL query on a table called `nyc_taxi` located in the lakehouse `lab140lakehouse`. The query selects all columns `(*)` from the table and limits the result to the first 1000 rows with the `LIMIT 1000` clause. The result of the query is then stored in a PySpark DataFrame called `df`.

`display(df)` - the `display()` function is used to visualize the contents of a DataFrame in a tabular format. In this case, it visualizes the contents of the df DataFrame created in the previous line.

In [ ]:
df = spark.sql("SELECT * FROM lab140lakehouse.nyc_taxi LIMIT 1000")

display(df)

Alternatively, you can use the %%sql magic in a notebook to run SQL statements.

In [ ]:
%%sql
SELECT * FROM lab140lakehouse.nyc_taxi LIMIT 1000

The code `df.show(5)` is used to display the first five rows of a DataFrame called df. This is a useful function when working with large datasets to quickly inspect the data and ensure that it has been loaded correctly. The number inside the parenthesis specifies the number of rows to display. In this case, the function call displays the first five rows of the DataFrame `df`. Each row is displayed as a separate line and the columns are separated by vertical bars.

In [ ]:
df.select("vendorID", "tripDistance", "fareAmount", "tipAmount").show(5)

When working with data, one of the initial tasks is to read it into the environment for analysis. Once the data is loaded, basic analysis such as filtering, sorting, and aggregating can be performed. However, as the scale and complexity of the data increase, there is a need for more advanced data engineering scenarios such as data cleansing, transformation, and aggregation. 

# 4. Data aggregation, summarization and correlation


In this scenario, the data engineer could aggregate and summarize the data to provide insights into the overall trends and patterns in the dataset. For example, they could group the data by some columns (such as VendorID or RatecodeID) and calculate some summary statistics for the numerical columns (such as average fare_amount or total trip_distance). This could involve using Spark's built-in aggregation functions (such as groupBy and agg) to perform these calculations.

The code calculates the average fare amount per month by grouping the DataFrame df by year and month of the lpep_pickup_datetime column. It uses the avg function from the [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html) module to calculate the average fare amount and aliases the resulting column as "average_fare". The resulting DataFrame average_fare_per_month is sorted by year and month and is displayed using the display function. Finally, the code saves the results to a new delta table named "average_fare_per_month" using the write function with "delta" format, and "overwrite" mode.

In [ ]:
from pyspark.sql.functions import col, year, month, dayofmonth, avg

df = spark.read.table("nyc_taxi")

# Calculate average fare amount per month
average_fare_per_month = (
    df
    .groupBy(year("lpepPickupDatetime").alias("year"), month("lpepPickupDatetime").alias("month"))
    .agg(avg("fareAmount").alias("average_fare"))
    .orderBy("year", "month")
)
display(average_fare_per_month)

# Save the results to a new delta table
average_fare_per_month.write.format("delta").mode("overwrite").saveAsTable("average_fare_per_month")

## Refresh Lakehouse explorer

<mark>Now, please follow along with the instructor.
</mark>

## Scatter chart

This code snippet demonstrates how to create a scatter plot using Matplotlib in Python. The code assumes that the Spark DataFrame df contains the columns fare_amount and trip_distance. First, the Spark DataFrame is converted to a Pandas DataFrame using the toPandas() function. Then, a scatter plot is created using ax.scatter() function. The x and y arguments of the scatter() function represent the variables to be plotted on the x- and y-axes, respectively. The alpha argument controls the transparency of the points in the scatter plot. The axis labels and title are set using the ax.set_xlabel(), ax.set_ylabel(), and ax.set_title() functions. Finally, the plot is displayed using the plt.show() function. This code can be used to visualize the correlation between fare amount and trip distance in the DataFrame.


In [ ]:
import matplotlib.pyplot as plt

# convert Spark DataFrame to Pandas DataFrame
df_pd = df.select(['fareAmount', 'tripDistance']).toPandas()

# create scatter plot
fig, ax = plt.subplots()
ax.scatter(x=df_pd['tripDistance'], y=df_pd['fareAmount'], alpha=0.5)

# set axis labels and title
ax.set_xlabel('Trip Distance')
ax.set_ylabel('Fare Amount')
ax.set_title('Correlation between Fare Amount and Trip Distance')

# show the plot
plt.show()


# 5. Custom libraries & advanced visualisation

Libraries provide reusable code that Data Engineers may want to include in their Spark application.

Each workspace comes with a pre-installed set of libraries available in the Spark run-time and available to be used immediately in the notebook or Spark job definition. 

Based on the user scenarios and specific needs, you can include other libraries. There are two types of libraries you may want to include:

- Feed library: Feed libraries are the ones that come from public sources or repositories. You can install Python feed libraries from PyPI and Conda by specifying the source in the Library Management portals. You can also use a Conda environment specification .yml file to install libraries.

- Custom library: Custom libraries are the code built by you or your organization. .whl, .jar and .tar.gz can be managed through Library Management portals. Note that .tar.gz is only supported for R language, please use .whl for Python custom libraries.

| **Library name** | **Workspace update** | **In-line installation** |
|---|---|---|
| **Python Feed (PyPI & Conda)** | Supported | Supported |
| **Python Custom (.whl)** | Supported | Supported |
| **R Feed (CRAN)** | Not Supported | Supported |
| **R custom (.tar.gz)** | Supported | Supported |
| **Jar** | Supported | Not Supported |

## Install library

The code line `pip install altair` is used to install the Python package "Altair" via the package manager "pip". Altair is a library for creating interactive visualizations in Python.

"Pip" is a package manager for Python that allows users to easily install, manage, and update Python packages (libraries) from the Python Package Index (PyPI) and other package repositories. Pip can be used to install packages globally on the system or locally in a specific virtual environment.



In [ ]:
pip install altair

## Create custom visualisation with a new library

First, the code imports the Altair library using the alias "alt". Next, the code uses Spark SQL to select all columns from the "nyc_taxi_cleansed" table in the "lab140lakehouse" database, limiting the result to the first 5000 rows. The resulting DataFrame is then converted to a Pandas DataFrame using the toPandas method.

The alt.Chart method is then called with the Pandas DataFrame as the data source, and the mark_point method is used to specify that the chart should use points as the visual mark. The encode method is then used to specify the encoding for the x-axis, y-axis, and color of the points, as well as the tooltip values. The x-axis is mapped to the "tripDistance" column, the y-axis is mapped to the "fareAmount" column, and the color of the points is mapped to the "paymentType" column, which is treated as a categorical variable. The tooltip displays the "tripDistance", "fareAmount", and "paymentType" columns for each point.

Finally, the interactive method is called to enable interactivity in the resulting visualization, allowing the user to zoom, pan, and view tooltip information when hovering over points in the scatter plot.

In [ ]:
import altair as alt

df = spark.sql("SELECT * FROM lab140lakehouse.nyc_taxi LIMIT 5000")

data = df.toPandas()

alt.Chart(data).mark_point().encode(
    x='tripDistance',
    y='fareAmount',
    color='paymentType:N',
    tooltip=['tripDistance', 'fareAmount', 'paymentType']
).interactive()

# 6. Shortcuts and final table 

Shortcuts in a lakehouse allow users to reference data without copying it. It unifies data from different lakehouses, workspaces, or external storage, such as ADLS Gen2 or AWS S3. You can quickly make large amounts of data available in your lakehouse locally without the latency of copying data from the source.

## Create shortcut to external ADLS Gen2

To create a shortcut, open Lakehouse Explorer and select where to place the shortcut under Tables or Files. Creating a shortcut to Delta formatted table under Tables in Lakehouse Explorer will automatically register it as a table, enabling data access through Spark, SQL endpoint, and default dataset. Spark can access shortcuts in Files for data science projects or for transformation into structured data.

**Objective: In this step, we aim to merge two datasets: the `nyc_taxi` delta table that currently resides in our lakehouse, and an external dataset located in ADLS Gen 2 that contains information about discounts offered on specific days. The final table will reflect all records from the `nyc_taxi` dataset with an additional column from the discount dataset, allowing us to see the total discount value per vendor for a given day. This will enable us to gain insights into how vendors offer discounts and how it impacts their revenue.**

<mark>Now, please follow along with the instructor.
</mark>


Connection settings:
- URL: `https://buildlab140ekot.dfs.core.windows.net/`
- Connection: Create new connection
- Connection name: NewConnectionToADLS
- Authentication kind: `Shared Access Signature (SAS)`
- SAS token: `?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwlacupx&se=2023-06-01T10:49:05Z&st=2023-05-04T02:49:05Z&spr=https,http&sig=0e0%2BlbFhxx3lcyz79VF272PLEzd0UdyMD348iNvBvQQ%3D`


**If you encounter the error message "The specified connection name already exists. Try choosing a different name", please make sure that the name you choose for the connection is unique.**

Then, for Shortcut settings:
- Name: `SideLoadingDiscountData`
- URL: confirm it's `https://buildlab140ekot.dfs.core.windows.net/`
- Sub Path `/data`

## Load new data

<mark>Now, please follow along with the instructor as they demonstrate.
</mark>


In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/SideLoadingDiscountData/Generated-NYC-Taxi-Green-Discounts.csv")
# df now is a Spark DataFrame containing CSV data from "Files/SideLoadingDiscountData/Generated-NYC-Taxi-Green-Discounts.csv".
display(df)

## Unpivot sideloaded data

The import pandas as pd line imports the Pandas library and assigns it an alias pd.

Melt the discounts DataFrame: The pd.melt() function is used to convert the discouts_df PySpark DataFrame to a long format by converting date columns into rows. First, discouts_df.toPandas() is used to convert the PySpark DataFrame to a Pandas DataFrame. Then, pd.melt() takes the Pandas DataFrame, uses 'VendorID' as the identifier variable (id_vars), sets the 'date' as the variable name (var_name), and 'discount' as the value name (value_name). The melted DataFrame is stored in discouts_pd_df.

Convert the melted DataFrame to a PySpark DataFrame: The spark.createDataFrame() function is used to convert the melted Pandas DataFrame discouts_pd_df back to a PySpark DataFrame, which is stored in the discounts_spark_df variable.

In [ ]:
import pandas as pd

# Melt discouts_df to long format
discouts_pd_df = pd.melt(df.toPandas(), id_vars=['VendorID'], var_name='date', value_name='discount')

discounts_spark_df = spark.createDataFrame(discouts_pd_df)

display(discounts_spark_df)

## Prepare data for join

In [ ]:
from pyspark.sql.functions import to_date

nyc_taxi_df = spark.sql("SELECT * FROM lab140lakehouse.nyc_taxi")

nyc_taxi_df = nyc_taxi_df.withColumn("date", to_date("lpepPickupDatetime"))

display(nyc_taxi_df)

## Join two datasets and save result

In [ ]:
from pyspark.sql.functions import col

# Create aliases for your DataFrames
df1_alias = nyc_taxi_df.alias("df1")
df2_alias = discounts_spark_df.alias("df2")

# Define the join condition using the aliases
join_condition = [col("df1.vendorID") == col("df2.VendorID"), col("df1.date") == col("df2.date")]

# Perform the join using the aliases
result_df = df1_alias.join(df2_alias, join_condition, how='inner')  # You can use other join types like 'left', 'right', 'outer', etc.

# Select only the desired columns
result_df = result_df.select("df1.vendorID", "df1.lpepPickupDatetime", "df2.discount")

display(result_df)

# Save the results to a new delta table
result_df.write.format("delta").mode("overwrite").saveAsTable("nyc_taxi_with_discounts")

## Refresh Lakehouse explorer  

<mark>Now, please follow along with the instructor.</mark>

### Remember to refresh lakehouse (Lakehouse explorer on the left) to see the new Delta table

<!-- ![image-alt-text](https://github.com/ekote/azure-architect/blob/master/images/4.gif?raw=true) -->

# 7. Use T-SQL to create VIEW

You can work with the data in the lakehouse in two modes:

1. Lake mode enables you to add and interact with tables, files, and folders in the Lakehouse.
2. **SQL Endpoint enables you to use SQL to query the tables in the lakehouse and manage its relational data model. It allows you to run Transact-SQL statements to query, filter, aggregate, and otherwise explore data in lakehouse tables.**

Project Trident's data warehouse experience allows you to transition from the lake view of the Lakehouse (which supports data engineering and Apache Spark) to the SQL experiences that a traditional data warehouse would provide.

In the data warehouse experience, you'll model data using **tables and views**, **run T-SQL to query data across the data warehouse and Lakehouse**, use **T-SQL to perform DML operations on data inside the data warehouse**, and **serve reporting layers like Power BI**.

<mark>Now, please follow along with the instructor..
</mark>

# 8. Power BI

The data in your lakehouse tables is included in a dataset that defines a relational model for your data. You can edit this dataset, defining custom measures, hierarchies, aggregations, and other elements of a data model. You can then use the dataset as the source for a Power BI report that enables you to visualize and analyze the data.

You can leverage the **DirectLake** feature to create Power BI datasets directly on top of your data stored in the Lakehouse. DirectLake enhances query performance when dealing with large data volumes and seamlessly integrates with Lakehouse workloads that read and write Parquet files. By combining the data visualization capabilities of Power BI with the centralized storage and tabular schema of a data lakehouse, you can implement an end-to-end analytics solution on a single platform.

**Project Trident enables you to visualize** the results of a single query or your entire data warehouse, **without leaving the data warehouse experience**. Exploring data while you work to ensure you have all the necessary data and transformations for your analysis is particularly useful.

Use the **Visualize button** to create a new Power BI report from the results of your query. Creating a new report with the results of your query will open a Power BI window.

You can also use the **New report button** to create a new Power BI report from the contents of your entire data warehouse. Using the New report button opens the Power BI service experience where you can build and save your report for use by the business.


<mark>Now, please follow along with the instructor.
</mark>



## DirectLake vs DirectQuery in Power BI

Power BI is natively integrated in the whole Project Trident experience. This native integration brings a unique mode, called DirectLake, of accessing the data from the lakehouse to provide the most performant query and reporting experience. DirectLake mode is a groundbreaking new engine capability to analyze very large datasets in Power BI. The technology is based on the idea of loading parquet-formatted files directly from a data lake without having to query a data warehouse or lakehouse endpoint, and without having to import or duplicate data into a Power BI dataset. DirectLake is a fast path to load the data from the data lake straight into the Power BI engine, ready for analysis.

In traditional DirectQuery mode, the Power BI engine queries the data directly from the data source every time it's queried and hence query performance depends on the speed data can be retrieved from the data source. This method avoids having to copy the data; any changes at the source are immediately reflected in the query results while in the import mode. And yet performance is better because the data is readily available in memory without having to query the data source each time. However, the Power BI engine must first copy the data into the dataset at refresh time. Any changes at the source are only picked up during the next data refresh.

DirectLake mode now eliminates this import requirement by loading the data files directly into memory. Because there's no explicit import process, it's possible to pick up any changes at the source as they occur, thus combining the advantages of DirectQuery and import mode while avoiding their disadvantages. DirectLake mode is therefore the ideal choice for analyzing very large datasets and datasets with frequent updates at the source.


# [EXTRA] Exploratory data analysis (EDA)



In this scenario, the data engineer could perform some data cleaning and transformation tasks to prepare the data for downstream analysis. 

Objective: **Cleanse the data and filter out invalid records for further analysis.**

In this scenario, we aim to demonstrate how data engineers can perform data cleansing and filtering on a large dataset. We begin by loading the data from the source and then filter out records where the trip distance and fare amount are less than or equal to zero, which are invalid records.

Next, we cleanse the data by converting the `store_and_fwd_flag` column to a boolean type, and converting the `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns to timestamp types. Finally, we write the cleansed data to the destination in the parquet format.

This scenario demonstrates the importance of data cleansing and filtering to ensure the data is accurate and valid before proceeding with further analysis.

In [ ]:
from pyspark.sql.functions import col, when

# Load data from source
df = spark.read.load("Tables/nyc_taxi", header=True, inferSchema=True)
df_count = df.count()

# Remove invalid records
df = df.filter((col("tripDistance") > 0) & (col("fareAmount") > 0))
df_count_after_clearning = df.count()

number_of_deleted_records = df_count - df_count_after_clearning

print(f"Removed {number_of_deleted_records} records")

# # Cleanse data
df = df.withColumn("storeAndFwdFlag", when(col("storeAndFwdFlag") == "Y", True).otherwise(False))
df = df.withColumn("lpepPickupDatetime", col("lpepPickupDatetime").cast("timestamp"))
df = df.withColumn("lpepDropoffDatetime", col("lpepDropoffDatetime").cast("timestamp"))

# Display cleansed data to destination
display(df)

# Write cleansed data to destination
df.write.format("delta").mode("overwrite").saveAsTable("nyc_taxi_cleansed")


Exploratory data analysis (EDA) is a common scenario for data engineers. EDA is the process of analyzing and understanding data to gain insights, identify patterns, and develop hypotheses for further investigation. In data engineering, EDA is often done to identify data quality issues, anomalies, or other problems that need to be addressed before data can be used for analysis or modeling. EDA can also help data engineers to understand the relationships between different data sources and determine the best way to join or transform them.

`df.count()` is a Spark DataFrame API function that returns the number of rows in the DataFrame. It is a convenient way to quickly determine the size of the DataFrame without having to iterate over all the rows manually. The function is an action in Spark, meaning it triggers a computation that counts the number of rows in the DataFrame and returns the result. It is useful for getting a quick overview of the data size and checking if any rows are missing or dropped during data processing. However, it should be used with caution on large datasets, as it can be a costly operation that requires significant computational resources.

In [ ]:
# Load data from source
df = spark.read.load("Tables/nyc_taxi_cleansed", header=True, inferSchema=True)

# Count the number of rows 
df.count()

# to simple - to remove

`df.dtypes` is an attribute of a DataFrame object that returns a list of tuples containing the column names and their corresponding data types. The data types are represented using the Spark SQL DataType class, which is a set of classes for representing data types in Spark SQL.

In [ ]:
# Display the data types of the columns.

df.dtypes

# to simple - to remove

The code imports the col function from `pyspark.sql.functions` and uses it to select the `"vendorID"` column from the Spark DataFrame `df`. The `groupBy()` function is then called on the resulting column object to group the DataFrame by the distinct values in the `"vendorID"` column. The `count()` function is then applied to the resulting grouped DataFrame to calculate the number of records in each group. Finally, the `show()` function is used to display the resulting DataFrame on the console.

In [ ]:
# Group the data by 'VendorID' and count the number of rows in each group. 

from pyspark.sql.functions import col

df.groupBy(col("vendorID")).count().show()

The code reads the Spark DataFrame `df` which contains information about NYC taxi trips. The code uses the `'min'` and `'max'` functions from PySpark to select the earliest and latest pickup dates respectively. These dates are stored in the variables `'oldest_day'` and `'latest_day'`. The `'collect'` function is then used to retrieve these values and they are printed to the console using the `'print'` function. The output displays the earliest and latest pickup dates in the dataset.

In [ ]:
# Retrieve information about the earliest and latest pickup dates in the dataset.

from pyspark.sql.functions import min, max

oldest_day = df.select(min("lpepPickupDatetime")).collect()[0][0]
latest_day = df.select(max("lpepDropoffDatetime")).collect()[0][0]

print("Oldest pickup date: ", oldest_day)
print("Latest pickup date: ", latest_day)

This code uses the PySpark `date_format` function to group the `df` DataFrame by the year, month, and day of the `lpep_pickup_datetime` column, and then counts the number of occurrences for each date.

`date_format` is a PySpark SQL function used to format the date or timestamp column to the specified format. In this code, the format used is `yyyy-MM-dd`. The alias `pickup_date` is assigned to the formatted date column, and the DataFrame is grouped by this column using the `groupby()` method. The `count()` method is then applied to count the number of occurrences of each pickup_date. Finally, the result is displayed using the `show()` method.

In [ ]:
from pyspark.sql.functions import date_format

# group by year, month and day of lpepPickupDatetime
df_grouped = df.groupby(date_format('lpepPickupDatetime', 'yyyy-MM-dd').alias('pickup_date')).count()

# show the result
df_grouped.show()

This code computes the minimum and maximum values of the fare_amount column in the Spark DataFrame df. It uses the `min()` and `max()` functions from the `pyspark.sql.functions` module to compute the minimum and maximum values, respectively. The `alias()` method is used to rename the resulting columns as `"min"` and `"max"`. Finally, the `show()` method is used to display the resulting DataFrame with two columns `"min"` and `"max"`, showing the minimum and maximum values of the `fare_amount column`.

In [ ]:
# min max values of target feature "fare_amount"

df.select(min('fareAmount').alias('min'), max('fareAmount').alias('max')).show()

This code is performing descriptive statistical analysis on the `"fare_amount"` column of a Spark DataFrame named `"df"`. Specifically, it is using the `describe()` method of the DataFrame to compute summary statistics including `count`, `mean`, `standard deviation`, `minimum`, and `maximum`.

The result of `describe()` is then converted to a Pandas DataFrame using the `toPandas()` method. This allows the statistics to be displayed in a more user-friendly table format, which includes the same summary statistics along with the 25th, 50th, and 75th percentiles. The resulting table provides insights into the central tendency and dispersion of the `"fare_amount"` variable, and can be useful for understanding the distribution of the data and identifying potential outliers.

In [ ]:
# General statistical characteristics of fare amount

df.select('fareAmount').describe().toPandas()

This code computes the approximate quantiles of the `'fare_amount'` column of the DataFrame `'df'` using the `'approxQuantile'` function from PySpark's SQL functions module. The function takes three arguments - the name of the column for which quantiles are to be computed, the list of quantile values to be returned, and a relative error value. In this case, the quantiles are 0.1, 0.25, 0.5, 0.75, and 0.9, and the relative error is set to 0.01. The function returns an array of approximate quantile values for the given column and quantile values.

In [ ]:
# quantiles

df.select('fareAmount').approxQuantile("fareAmount",[0.1, 0.25, 0.5, 0.75, 0.9], 0.01)

This code is used to plot the distribution of fare_amount using matplotlib library in Python. The fare_amount data is first extracted from the Spark DataFrame using the select function along with the F.col() function to extract the fare_amount column. The resulting DataFrame is then converted to a Pandas DataFrame using the toPandas() function. The fare_amount data is then plotted as a histogram using the hist() function from matplotlib. The number of bins for the histogram is set to 50 using the bins parameter. Finally, the title and axis labels for the plot are set using the title(), xlabel(), and ylabel() functions, and the plot is displayed using the show() function.

In [ ]:
import matplotlib.pyplot as plt
import pyspark.sql.functions as F

# Assuming your DataFrame is named `df`
fare_distribution = df.select(F.col('fareAmount')).toPandas()

# Plot histogram
plt.hist(fare_distribution, bins=50)
plt.title('Distribution of fareAmount')
plt.xlabel('fareAmount')
plt.ylabel('Frequency')
plt.show()

# [EXTRA] Medallion architecture

A Medallion architecture is a data design pattern used to organize data in a Lakehouse, with the goal of progressively improving the quality and structure of the data as it flows through each layer of the architecture, starting from the Bronze layer, then to the Silver layer, and finally to the Gold layer.

![image-alt-text](https://techcommunity.microsoft.com/t5/image/serverpage/image-id/243714iAF59794D11862CC4/image-dimensions/521x259?v=v2)

This incremental and progressive improvement enables you to maintain data quality and structure while also improving data processing performance. Medallion architectures are sometimes referred to as "multi-hop" architectures because data flows through multiple layers.

One of the main benefits of a Lakehouse architecture is that it provides a simple data model that is easy to understand and implement. Additionally, it enables incremental ETL (extract, transform, load) operations, which means you can add new data to the Lakehouse in a scalable and manageable way.

Another benefit of a Lakehouse architecture is that it allows you to recreate your tables from raw data at any time. This is possible because Delta Lake provides ACID transactions and time travel capabilities, allowing you to track changes to your data and easily roll back to previous versions if necessary.

Read more [here](https://techcommunity.microsoft.com/t5/analytics-on-azure-blog/simplify-your-lakehouse-architecture-with-azure-databricks-delta/ba-p/2027272).


## Medallion architecture in Project Trident Lakehouse

After performing data cleaning and transformation on your Lakehouse data, you can save the resulting data back to another Lakehouse to reflect the "bronze->silver->gold" pattern.

Here's an example code snippet that shows how you can write data to another Lakehouse:

```python
# read data from the bronze Lakehouse
bronze_df = spark.read.table("bronze_lakehouse_name.lakehouse_table")

# perform data cleaning and transformation
# ...

# write the transformed data to the silver Lakehouse
transformed_df.write.format("delta").mode("overwrite").saveAsTable("silver_lakehouse_name.lakehouse_table")
```

In this example, we first read data from the bronze Lakehouse using the spark.read method. We then perform data cleaning and transformation on the bronze_df DataFrame. Finally, we write the transformed data to the silver Lakehouse using the transformed_df.write method, specifying the path to the silver Lakehouse and setting the save mode to "overwrite" to replace any existing data.

# Final thought

Project Trident lakehouses are designed to provide data engineers and analysts with the benefits of both data lake storage and a relational data warehouse. Apache Spark is a critical technology for big data analytics, and its support within Project Trident allows you to seamlessly integrate Spark's big data processing capabilities with the other data analytics and visualization tools available on the platform.

By using a lakehouse, you can create an end-to-end data analytics solution that includes data ingestion, transformation, modeling, and visualization. The lakehouse provides a unified and scalable platform for storing and managing data, allowing you to easily access and analyze both structured and unstructured data. Additionally, the platform's built-in security and compliance features help ensure that your data is always secure and compliant with industry standards.

Thank you!!

Project Trident Team
